In [1]:
%load_ext autoreload
%autoreload 2

import sys
import os
import time

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from scipy.ndimage.interpolation import map_coordinates
from skimage.exposure import rescale_intensity
from skimage.transform import rotate

sys.path.append(os.environ['REPO_DIR'] + '/utilities')
from utilities2015 import *
from metadata import *
from data_manager import *
from learning_utilities import *
from distributed_utilities import *
from visualization_utilities import *

Setting environment for Precision WorkStation


No vtk
No vtk/usr/local/lib/python2.7/dist-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


# Plot histogram over whole images.

In [2]:
stack_versions = {'MD585': 'gray',
'MD589': 'gray',
'MD594': 'gray',
'MD635': 'Ntb',
'MD653': 'Ntb',
 'MD652': 'Ntb',
 'MD642': 'Ntb',
 'MD657': 'Ntb',
 'MD658': 'Ntb',
 'MD661': 'Ntb',
 'MD662': 'Ntb' 
}

In [ ]:
# for stack in ['MD585', 'MD589', 'MD594'] + ['MD635'] + ['MD653', 'MD652', 'MD642', 'MD657', 'MD658'] + ['MD661', 'MD662']:
for stack in ['MD635'] + ['MD653', 'MD652', 'MD642', 'MD657', 'MD658'] + ['MD661', 'MD662']:

    structure = '7N'
    
    if stack in ['MD585', 'MD589', 'MD594'] + ['MD658'] + ['MD635']:
        local_region_bboxes_all_sections = get_local_regions(stack, margin_um=200, by_human=True ,
                                                         structures=['7N_L'])
    elif stack in ['MD653', 'MD652', 'MD642', 'MD657']:
        local_region_bboxes_all_sections = get_local_regions(stack, margin_um=200, by_human=False, level=0.5, 
                                                     structures=['7N_L'],
                                                    detector_id_f=13)
    elif stack in ['MD661', 'MD662']:
        local_region_bboxes_all_sections = get_local_regions(stack, margin_um=200, by_human=False, level=0.5, 
                                                     structures=['7N_L'],
                                                    detector_id_f=1)
    else:
        raise
        
    for name_s, bboxes_all_sections in local_region_bboxes_all_sections.iteritems():
        for sec, bbox in bboxes_all_sections.iteritems():

#     for sec in metadata_cache['valid_sections'][stack]:
        
#             try:
            print sec

            NorF = DataManager.get_brightfield_or_fluorescence(stack=stack, sec=sec)

            lossless_img = DataManager.load_image_v2(stack=stack, prep_id=2, section=sec, version=stack_versions[stack])

            tb_mask = DataManager.load_thumbnail_mask_v3(stack=stack, prep_id=2, section=sec)
            lossless_mask = rescale_by_resampling(tb_mask, 32)

            if NorF == 'N':
                counts, bins = np.histogram(lossless_img[lossless_mask], bins=np.linspace(0, 256, 257));
            elif NorF == 'F':
                counts, bins = np.histogram(lossless_img[lossless_mask], bins=np.linspace(0, 3500, 300));
            else:
                raise

            vmax = lossless_img[lossless_mask].max()
            vmin = lossless_img[lossless_mask].min()

            fig = plt.figure(figsize=(10, 5));
            plt.bar(bins[:-1], counts, align='center')
            if NorF == 'N':
                plt.xlim([-1, 256]);
                plt.xticks(np.r_[bins[::50], vmin, vmax])
            else:
                plt.xlim([-1, 3500]);
                plt.xticks(np.r_[bins[::50], vmin, vmax])
            plt.axvline(x=vmax, c='r');
            plt.axvline(x=vmin, c='r');
            plt.title('Histogram (linear)');
            plt.xlabel('Pixel intensity');
    #                 plt.show();

            plot_fp = os.path.join(ROOT_DIR, 'raw_image_histograms_fullImage', stack, '%s_%d%s.png' % (stack, sec, NorF))
            create_parent_dir_if_not_exists(plot_fp)
            plt.savefig(fname=plot_fp, bbox_inches='tight');
            plt.close(fig);
            upload_to_s3(plot_fp)

            counts_log = np.log(counts)

            fig = plt.figure(figsize=(10, 5));
            plt.bar(bins[:-1], counts_log, width=np.diff(bins)[0]*.8, align='center');
            if NorF == 'N':
                plt.xlim([-1, 256]);
                plt.xticks(np.r_[bins[::50], vmin, vmax])
            else:
                plt.xlim([-1, 3500]);
                plt.xticks(np.r_[bins[::50], vmin, vmax])

            plt.axvline(x=vmax, c='r');
            plt.axvline(x=vmin, c='r');
            plt.title('Histogram (log)');
            plt.xlabel('Pixel intensity');
    #                 plt.show();

            plot_fp = os.path.join(ROOT_DIR, 'raw_image_histograms_log_fullImage', stack, '%s_%d%s.png' % (stack, sec, NorF))
            create_parent_dir_if_not_exists(plot_fp)
            plt.savefig(fname=plot_fp, bbox_inches='tight');
            plt.close(fig);
            upload_to_s3(plot_fp)
#             except:
#                 pass

In [2]:
stack_versions = {'CHATM2': 'Ntb'}

In [3]:
nissl_bins = np.linspace(0, 256, 257)
ntb_bins = np.linspace(0, 65536, 300)

In [8]:
for stack in ['CHATM2']:

#     for sec in metadata_cache['valid_sections'][stack][::10]:
    for sec in range(150, 170):
        
        try:
        
            print sec

            NorF = DataManager.get_brightfield_or_fluorescence(stack=stack, sec=sec)

            lossless_img = DataManager.load_image_v2(stack=stack, prep_id=2, section=sec, version=stack_versions[stack])

            tb_mask = DataManager.load_thumbnail_mask_v3(stack=stack, prep_id=2, section=sec)
            lossless_mask = rescale_by_resampling(tb_mask, 32)

            if NorF == 'N':
                counts, bins = np.histogram(lossless_img[lossless_mask], bins=nissl_bins);
            elif NorF == 'F':
                counts, bins = np.histogram(lossless_img[lossless_mask], bins=ntb_bins);
            else:
                raise

            vmax = lossless_img[lossless_mask].max()
            vmin = lossless_img[lossless_mask].min()

#             fig = plt.figure(figsize=(10, 5));
#             plt.bar(bins[:-1], counts, align='center')
#             if NorF == 'N':
#                 plt.xlim([-1, nissl_bins[-1]]);
#                 plt.xticks(np.r_[bins[::50], vmin, vmax], rotation=60)
#             else:
#                 plt.xlim([-1, ntb_bins[-1]]);
#                 plt.xticks(np.r_[bins[::50], vmin, vmax], rotation=60)
#             plt.axvline(x=vmax, c='r');
#             plt.axvline(x=vmin, c='r');
#             plt.title('Histogram (linear)');
#             plt.xlabel('Pixel intensity');
#             plt.show();

#             plot_fp = os.path.join(ROOT_DIR, 'raw_image_histograms_fullImage', stack, '%s_%d%s.png' % (stack, sec, NorF))
#             create_parent_dir_if_not_exists(plot_fp)
#             plt.savefig(fname=plot_fp, bbox_inches='tight');
#             plt.close(fig);
#             upload_to_s3(plot_fp)

            counts_log = np.log(counts)

            fig = plt.figure(figsize=(10, 5));
            plt.bar(bins[:-1], counts_log, width=np.diff(bins)[0]*.8, align='center');
            if NorF == 'N':
                plt.xlim([-1, nissl_bins[-1]]);
                plt.xticks(np.r_[bins[::50], vmin, vmax], rotation=60)
            else:
                plt.xlim([-1, ntb_bins[-1]]);
                plt.xticks(np.r_[bins[::50], vmin, vmax], rotation=60)

            plt.axvline(x=vmax, c='r');
            plt.axvline(x=vmin, c='r');
            plt.title('Histogram (log)');
            plt.xlabel('Pixel intensity');
#             plt.show();

            plot_fp = os.path.join(ROOT_DIR, 'raw_image_histograms_log_fullImage', stack, '%s_%d%s.png' % (stack, sec, NorF))
            create_parent_dir_if_not_exists(plot_fp)
            plt.savefig(fname=plot_fp, bbox_inches='tight');
            plt.close(fig);
            upload_to_s3(plot_fp)
            
        except Exception as e:
            print e
            pass

150


Not using image_cache.


/media/yuncong/BstemAtlasData/CSHL_data_processed/CHATM2/CHATM2_prep2_lossless_Ntb/CHATM2_slide50_2018_01_22-S3_prep2_lossless_Ntb.tif
aws s3 cp "/home/yuncong/raw_image_histograms_log_fullImage/CHATM2/CHATM2_150F.png" "s3://mousebrainatlas-data/raw_image_histograms_log_fullImage/CHATM2/CHATM2_150F.png"
0
151


Not using image_cache.


/media/yuncong/BstemAtlasData/CSHL_data_processed/CHATM2/CHATM2_prep2_lossless_Ntb/CHATM2_slide51_2018_01_22-S1_prep2_lossless_Ntb.tif
aws s3 cp "/home/yuncong/raw_image_histograms_log_fullImage/CHATM2/CHATM2_151F.png" "s3://mousebrainatlas-data/raw_image_histograms_log_fullImage/CHATM2/CHATM2_151F.png"
0
152


...

/media/yuncong/BstemAtlasData/CSHL_data_processed/CHATM2/CHATM2_prep2_lossless_Ntb/CHATM2_slide56_2018_01_22-S2_prep2_lossless_Ntb.tif
aws s3 cp "/home/yuncong/raw_image_histograms_log_fullImage/CHATM2/CHATM2_167F.png" "s3://mousebrainatlas-data/raw_image_histograms_log_fullImage/CHATM2/CHATM2_167F.png"
0
168


Not using image_cache.


/media/yuncong/BstemAtlasData/CSHL_data_processed/CHATM2/CHATM2_prep2_lossless_Ntb/CHATM2_slide56_2018_01_22-S3_prep2_lossless_Ntb.tif
aws s3 cp "/home/yuncong/raw_image_histograms_log_fullImage/CHATM2/CHATM2_168F.png" "s3://mousebrainatlas-data/raw_image_histograms_log_fullImage/CHATM2/CHATM2_168F.png"
0
169


Not using image_cache.


/media/yuncong/BstemAtlasData/CSHL_data_processed/CHATM2/CHATM2_prep2_lossless_Ntb/CHATM2_slide57_2018_01_22-S1_prep2_lossless_Ntb.tif
aws s3 cp "/home/yuncong/raw_image_histograms_log_fullImage/CHATM2/CHATM2_169F.png" "s3://mousebrainatlas-data/raw_image_histograms_log_fullImage/CHATM2/CHATM2_169F.png"
0
